In [1]:
import json
from pathlib import Path
from datetime import datetime, timezone
import re

In [2]:
exported_restrictions = Path('Road_restrictions_11_13_25.json')

with exported_restrictions.open("r", encoding="utf-8") as f:
    new_restrictions = json.load(f)
    
print(type(new_restrictions))

<class 'dict'>


In [3]:
len(new_restrictions['features'])

4

In [4]:
from datetime import datetime

# Get the current local date and time
# Get the current time in UTC
now_utc = datetime.now(timezone.utc)

# Format using string format codes
current_datetime = now_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

#current_datetime

## Data Sources

- Create a list of dictionaries for each data source

In [ ]:
# TODO: We should manage the feed on a rolling basis. Open the prior verion of the feed, update restrictions or append, as appropriate
# rather than generating the entire thing anew
# this will result in a better 'update date' in the 'data sources' list

park_list = []
contact_list = []
contact_email_list = []

data_sources = []

for x in range(len(new_restrictions['features'])):

    #turn id into a string
    new_restrictions['features'][x]['id'] = str(new_restrictions['features'][x]['attributes']['OBJECTID'])
    
    #grab the data source info
    park_alpha = new_restrictions['features'][x]['attributes']['park_alpha']
    contact = new_restrictions['features'][x]['attributes']['contact_name']
    
    #skip if no contact name
    if contact is None:
        continue

    #manually add emails - delete when data updated with emails
    if new_restrictions['features'][x]['attributes']['park_alpha'] == "ZION":
        contact_email = "Lisa.White@nps.gov"
    else:
        contact_email = "steven_kattula@nps.gov"

    #create unique dictionary for each park of data_sources info
    this_data_source = {
        "data_source_id": park_alpha,
        "organization_name": park_alpha,
        "contact_name": contact,
        "contact_email": contact_email, 
        "update_date": str(current_datetime)   
        }
        
    # append this_data_source to data_sources - make sure no duplicate entries before adding to the list
    
    
    if this_data_source not in data_sources:
        data_sources.append(this_data_source)

In [6]:
data_sources

[{'data_source_id': 'ZION',
  'organization_name': 'ZION',
  'contact_name': 'Lisa White',
  'contact_email': 'Lisa.White@nps.gov',
  'update_date': '2025-11-14T17:28:17Z'},
 {'data_source_id': 'GRSM',
  'organization_name': 'GRSM',
  'contact_name': 'Katie Liming',
  'contact_email': 'steven_kattula@nps.gov',
  'update_date': '2025-11-14T17:28:17Z'}]

## Feed Info

- Create the initial block of the road restriction json that has the overall feed_info



In [7]:
feed_info_key = {   #this stuff look like it stays static except for the date which I can figure out how to update later
    'update_date' : str(current_datetime),
    'publisher' : "National Park Service",
    'contact_name' :"National Park Service" ,
    'contact_email' : "asknps@nps.gov",
    'version' : "4.0",
    'license' : "https://creativecommons.org/publicdomain/zero/1.0/",
    'data_sources' : data_sources}

feed_info_dict = {'feed_info': feed_info_key}

In [8]:
feed_info_key|{"type": "FeatureCollection"}

{'update_date': '2025-11-14T17:28:17Z',
 'publisher': 'National Park Service',
 'contact_name': 'National Park Service',
 'contact_email': 'asknps@nps.gov',
 'version': '4.0',
 'license': 'https://creativecommons.org/publicdomain/zero/1.0/',
 'data_sources': [{'data_source_id': 'ZION',
   'organization_name': 'ZION',
   'contact_name': 'Lisa White',
   'contact_email': 'Lisa.White@nps.gov',
   'update_date': '2025-11-14T17:28:17Z'},
  {'data_source_id': 'GRSM',
   'organization_name': 'GRSM',
   'contact_name': 'Katie Liming',
   'contact_email': 'steven_kattula@nps.gov',
   'update_date': '2025-11-14T17:28:17Z'}],
 'type': 'FeatureCollection'}

## Create Features

In [9]:
features = []
restriction_type_list = ["local_access_only", "no_trucks","reduced_width","reduced_height",
                         "reduced_length","reduced_weight", "axle_load_limit","gross_weight_limit",
                         "towing_prohibited","permitted_oversize_loads_prohibited","no_passing"]

for x in range(len(new_restrictions['features'])):

    #create lists for each restriction
    this_restriction = []
    this_lane = []
    road_names=[]

    #add in initial data features
    id = new_restrictions['features'][x]['attributes']['OBJECTID']
    park_alpha = new_restrictions['features'][x]['attributes']['park_alpha']
    road_names.append(new_restrictions['features'][x]['attributes']['road_names'])
    direction = 'undefined'
    description = new_restrictions['features'][x]['attributes']['additional_notes_for_travelers']
    
    # this direction field was removed from the ArcGIS database. We may add it back in later. Uncomment the following when that happens
    # direction = new_restrictions['features'][x]['attributes']['Direction']
    
    #TODO: open issue: most direction data missing - set to northbound if missing for now
    # if direction is None:
    #    direction = "undefined"
    
    
    #add in and clean the geometry
    geometry = new_restrictions['features'][x]['geometry']
    geometry["coordinates"] = geometry.pop("paths")[0]
    geometry["type"] = "LineString"
    
    reversed_geometry_coords = geometry['coordinates'][::-1]
 
    #create core details dictionary
    this_core_details = {
        "event_type": "restriction",
        "data_source_id": park_alpha,
        "road_names": road_names,
        "direction": direction,
        "description": description
    }

    #create restrictions list of dictionaries
    for y in restriction_type_list:
        restriction_value = new_restrictions['features'][x]['attributes'][y]
        if restriction_value not in [None, "", 0]:
            print(y) #debugging
            if y=="reduced_width":
                this_restriction_partial = {
                    "type": "reduced-width",
                    "value": round(float(new_restrictions['features'][x]['attributes']['reduced_width_value']), 1),
                    "unit": new_restrictions['features'][x]['attributes']['reduced_width_unit']
                }

            elif y=="reduced_height":
                this_restriction_partial = {
                    "type": "reduced-height",
                    "value": round(float(new_restrictions['features'][x]['attributes']['reduced_height_value']), 1),
                    "unit": new_restrictions['features'][x]['attributes']['reduced_height_unit']
                }

            elif y=="reduced_length":
                this_restriction_partial = {
                    "type": "reduced-length",
                    "value": round(float(new_restrictions['features'][x]['attributes']['reduced_length_value']), 1),
                    "unit": new_restrictions['features'][x]['attributes']['reduced_length_unit']
                }

            elif y=="reduced_weight":
                this_restriction_partial = {
                    "type": "reduced-weight",
                    "value": round(float(new_restrictions['features'][x]['attributes']['reduced_weight_value']), 1),
                    "unit": new_restrictions['features'][x]['attributes']['reduced_weight_unit']
                }
            else:
                this_restriction_partial = {
                    "type": y.replace("_","-")
                } 
                
            #append dictionaries to restriction list
            this_restriction.append(this_restriction_partial) 

    #create lane dictionary and append to lane list
    #for now, only one lane per restriction
    #TODO: open issue: lane order - for now, set to 1
    this_lane.append({
        "order": 1,
        "status": "open", 
        "type": "general",
        "restrictions": this_restriction
    })

    #create feature dictionary and append to features list
    this_feature = {
        "id": park_alpha,
        "type": "Feature",
        "properties": {"core_details": this_core_details, 
                       "restrictions": this_restriction, 
                       "lanes": this_lane},
        "geometry": geometry, 
        }
    
    # make a copy of the whole feature, then flip the geometry
    reversed_this_feature = this_feature  
    reversed_this_feature['geometry']['coordinates'] = reversed_geometry_coords
        
    #append to features list - make sure no duplicate entries before adding to the list
    if this_feature not in features:
        features.append(this_feature)
        features.append(reversed_this_feature)



reduced_width
reduced_height
reduced_length
reduced_weight
permitted_oversize_loads_prohibited
no_passing


## Create final output JSON

In [10]:
#append the 3 main parts together
updated_feed = {}
updated_feed = feed_info_dict | {"type": "FeatureCollection"} | {"features": features}

In [11]:
filename_date = re.sub(r'[^A-Za-z0-9._-]', '_', current_datetime)

filename = 'NPSRoadRestrictions_' + filename_date + '.json'

path = Path('restrictions_feeds') / filename

with path.open("w", encoding="utf-8") as f:
    json.dump(updated_feed, f, indent=2, ensure_ascii=False)